![Clothing Classifier Model](Clothing%20Classifier%20Model.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.


In [261]:
# Run the cells below first

In [262]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [263]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [264]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [266]:
print(type(train_data))
print(len(train_data))
print(type(train_data[0]))
print(len(train_data[0]))
print(type(train_data[0][0]))
print(type(train_data[0][1]))
print(train_data[0][0].shape)
#print(type(train_data[0][0][0]))
#print(train_data[0][0][0].shape)
#print(train_data[0][0][0][0].shape)
print(train_data[0][0][0][12][25])

<class 'torchvision.datasets.mnist.FashionMNIST'>
60000
<class 'tuple'>
2
<class 'torch.Tensor'>
<class 'int'>
torch.Size([1, 28, 28])
tensor(0.4667)


In [267]:
#print(set([train_data[i][1] for i in range(len(train_data
print(len(train_data.classes))

10


Input : 60000 images de taille 28x28, noir&blanc (?)
Label : int de 0 à 9


Model : 
 - Conv : 1 x 28x28 -> k x 28x28        WITHOUT PADDING:26
 - ReLU
 - MaxPool : k x 28x28 -> k x 14x14
 - Linear : k x 14x14 -> 10


In [268]:


dataloader_train = DataLoader(train_data, shuffle = True, batch_size = 100)



In [269]:
k = 16

model = nn.Sequential(
    nn.Conv2d(1, k, kernel_size = 3, stride=1, padding=1),   
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride = 2),
    nn.Flatten(),
    nn.Linear(k*14*14, 10)   # ????  pas de k ?!
)

#model(train_data[0][0])

In [270]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


model.train()

for epoch in range(1):
    for img, lbl in dataloader_train:
        optimizer.zero_grad()
        out = model(img)
        label = nn.functional.one_hot(torch.tensor(lbl), num_classes = 10)
        loss = criterion(out.double(), label.double())
        loss.backward()
        optimizer.step

    

In [271]:
#model.eval()

#predictions_vector = [model(img) for img, _ in test_data]
#predictions = [torch.max(pred,1)[1] for pred in predictions_vector]

#print(predictions)

In [272]:
acc = Accuracy(task='multiclass', num_classes = 10)
prec = Precision(task='multiclass', num_classes = 10, average=None)
rec = Recall(task='multiclass', num_classes = 10, average=None)

model.eval()

dataloader_test = DataLoader(test_data, batch_size=100, shuffle=False)

predictions = []
with torch.no_grad():
    for img, lbl in dataloader_test:
        label = nn.functional.one_hot(torch.tensor(lbl), num_classes = 10)
        pred_vect = model(img)
        _, pred = torch.max(pred_vect, 1)  #prediction comme entier
        predictions.append(pred)
        
        prec(pred_vect, label)    #(pred, lbl) pour des entier
        rec(pred_vect, label)
        acc(pred_vect, label)
#        acc(pred.item(), lbl)
    precision = prec.compute()
    recall = rec.compute()
    accuracy = acc.compute()

print(f"Precision: {precision} \n Recall: {recall} \n Accuracy : {accuracy}")
    

Precision: tensor([0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000]) 
 Recall: tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) 
 Accuracy : 0.0


In [275]:
#print(predictions)